In [2479]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib ipympl

Jan_return = pd.read_csv('Jan_return.csv')
Aug_return = pd.read_csv('Aug_return.csv')
Oct_return = pd.read_csv('Oct_return.csv')
Jan_return = Jan_return.drop(['Unnamed: 0'], axis=1)
Aug_return = Aug_return.drop(['Unnamed: 0'], axis=1)
Oct_return = Oct_return.drop(['Unnamed: 0'], axis=1)
Jan_predict_return = pd.read_csv('table_predict_return_Jan.csv')
Aug_predict_return = pd.read_csv('table_predict_return_Aug.csv')
Oct_predict_return = pd.read_csv('table_predict_return_Oct.csv')
Jan_predict_return = Jan_predict_return.drop(['Unnamed: 0'], axis=1)
Aug_predict_return = Aug_predict_return.drop(['Unnamed: 0'], axis=1)
Oct_predict_return = Oct_predict_return.drop(['Unnamed: 0'], axis=1)
#Jan_return = Jan_return.to_numpy()
#Aug_return = Aug_return.to_numpy()
#Oct_return = Oct_return.to_numpy()
#Jan_predict_return = Jan_predict_return.to_numpy()
#Aug_predict_return = Aug_predict_return.to_numpy()
#Oct_predict_return = Oct_predict_return.to_numpy()

In [2480]:
o = 9
p = 1000
q = 1200

In [2481]:

Jan_return = (1 + Jan_return)** o - 1 
Aug_return = ( Aug_return) * p
Oct_return = (Oct_return) * q 
x = np.cov(Jan_return.to_numpy().T)
y = np.cov(Aug_return.to_numpy().T)
z = np.cov(Oct_return.to_numpy().T)

In [2482]:
a = np.matmul(((1 + Jan_predict_return)**o -1).to_numpy(), np.linalg.inv(x))
b = np.matmul((( Aug_predict_return)*p ).to_numpy(), np.linalg.inv(y))
c = np.matmul(((Oct_predict_return)*q ).to_numpy(), np.linalg.inv(z))

In [2483]:
new_a = np.zeros((a.shape[0],))
new_b = np.zeros((b.shape[0],))
new_c = np.zeros((c.shape[0],))
for i in range(a.shape[0]):
    new_a[i] = np.dot(a[i], a[i])

for i in range(b.shape[0]):
    new_b[i] = np.dot(b[i], b[i])

for i in range(c.shape[0]):
    new_c[i] = np.dot(c[i], c[i])



In [2484]:
"""let the number larger than 1 be the avergae of the rest"""
new_a[new_a > 1] = np.mean(new_a[new_a < 1])
new_b[new_b > 1] = np.mean(new_b[new_b < 1])
new_c[new_c > 1] = np.mean(new_c[new_c < 1])

In [2485]:
print(new_a.mean())
print(new_b.mean())
print(new_c.mean())

9.436324749653754e-06
0.015121132150418255
0.004796919141489741


In [2486]:

date_a = np.linspace(30, 0, 671)
date_b = np.linspace(30, 0, 623)
date_c = np.linspace(30, 0, 647)

In [2487]:
new_a = new_a[1:] * date_a
new_b = new_b[1:] * date_b
new_c = new_c[1:] * date_c


a = np.exp(new_a)
b = np.exp(new_b)
c = np.exp(new_c)

In [2488]:
gamma = 0.5

policy_mean_coeff_Jan = -1 * np.matmul(np.matmul(np.linalg.inv(x), np.linalg.inv(x)), ((1 + Jan_predict_return)**o -1).to_numpy().T).T
policy_mean_coeff_Aug = -1 * np.matmul(np.matmul(np.linalg.inv(y), np.linalg.inv(y)), ((Aug_predict_return)*p ).to_numpy().T).T
policy_mean_coeff_Oct = -1 * np.matmul(np.matmul(np.linalg.inv(z), np.linalg.inv(z)), ((Oct_predict_return)*q ).to_numpy().T).T
policy_mean_coeff_Jan = policy_mean_coeff_Jan[1:]
policy_mean_coeff_Aug = policy_mean_coeff_Aug[1:]
policy_mean_coeff_Oct = policy_mean_coeff_Oct[1:]

policy_var_coeff_Jan = np.zeros((671, 9,9))
policy_var_coeff_Aug = np.zeros((623, 9,9))
policy_var_coeff_Oct = np.zeros((647, 9,9))

for i in range(671):
    policy_var_coeff_Jan[i] = 0.5 * gamma  * np.matmul(np.linalg.inv(x), np.linalg.inv(x)) * a[i]
for i in range(623):
    policy_var_coeff_Aug[i] = 0.5 * gamma  * np.matmul(np.linalg.inv(y), np.linalg.inv(y)) * b[i]
for i in range(647):
    policy_var_coeff_Oct[i] = 0.5 * gamma  * np.matmul(np.linalg.inv(z), np.linalg.inv(z)) * c[i]

In [2489]:
policy_mean_coeff_Jan[56]

array([ 8.28393252e-14,  1.43343283e-14,  6.54227169e-10, -1.46206630e-09,
        9.61202217e-10, -4.66585731e-10,  3.06247811e-10, -3.56217093e-13,
       -9.32478022e-13])

In [2490]:
policy_mean_coeff_Aug[23]

array([ 8.87465636e-05, -5.81296074e-06,  6.50479485e-07,  1.55057461e-06,
       -1.33054549e-06, -3.96994461e-07,  2.81371604e-09, -8.51913551e-05,
        2.54540591e-07])

In [2491]:
policy_mean_coeff_Oct[12]

array([ 1.62309508e-06,  8.65569087e-06,  3.32180998e-05,  5.14898623e-05,
       -8.59968601e-05, -4.16608381e-09, -1.38343079e-07, -1.32412671e-05,
        3.10393175e-08])

In [2504]:
Jan_wealth = [1]
Aug_wealth = [1]
Oct_wealth = [1]

for i in range(671):
    holdings = np.random.multivariate_normal(policy_mean_coeff_Jan[i] * (Jan_wealth[-1] - 1), policy_var_coeff_Jan[i])
    """if the absolute value of the holding is larger than 1, then set it to 0.5"""
    holdings[abs(holdings) > 0.5] = 0
    Jan_wealth.append(np.dot(holdings, Jan_return.iloc[i, :].to_numpy()) + Jan_wealth[-1])

for i in range(623):
    holdings = np.random.multivariate_normal(policy_mean_coeff_Aug[i] * (Aug_wealth[-1] - 1) , policy_var_coeff_Aug[i])
    holdings[abs(holdings) > 0.5] = 0
    profits = np.dot(holdings, Aug_return.iloc[i, :].to_numpy())
    if abs(profits) < 1: #leverage is constrained, so if the profit is too high, it is due to the unusual electricity prices
        Aug_wealth.append(profits + Aug_wealth[-1])
    else:
        Aug_wealth.append(Aug_wealth[-1])



for i in range(647):   #647
    holdings = np.random.multivariate_normal(policy_mean_coeff_Oct[i] * (Oct_wealth[-1] - 1), policy_var_coeff_Oct[i])
    holdings[abs(holdings) > 0.5 ] = 0
    profit = np.dot(holdings, Oct_return.iloc[i, :].to_numpy())
    if abs(profit) < 1:
        Oct_wealth.append(profit + Oct_wealth[-1])
    else:
        Oct_wealth.append(Oct_wealth[-1])


#plt.figure(figsize=(10, 5))
#plt.plot(Jan_wealth, label = 'Jan')
#plt.show()

#plt.figure(figsize=(10, 5))
#plt.plot(Aug_wealth, label='Aug')
#plt.show()
#plt.savefig('Aug_wealth_second.png')


#plt.figure(figsize=(10, 5))
#plt.plot(Oct_wealth, label='Oct')
#plt.show()
#plt.savefig('Oct_wealth_second.png')
